In [76]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
import numpy as np
import glob
import os
import scipy.cluster.hierarchy as sch
import matplotlib.pylab as plt
import json
import shutil

In [77]:
#图片存放地址
class_path = "/home/arvin/flower/class/0"
#获取本类中排过序的文件名列表
if os.path.exists(class_path):
    files = sorted(glob.glob(os.path.join(class_path, '*.jpg')))
else:
    print "dict not exists!"
    os._exit()

In [78]:
#备份图片到class_pass/source
shutil.copytree(class_path, os.path.join(class_path,'source'))

In [79]:
def max_pool(data):
    '''最大池化，将维度减半
       输入：np.array , shape (1,x),x为偶数
       输出：np.array , shape (1,x/2)
    '''
    ls = []
    i=0
    while i< data.shape[1]:
        j = i + 1
        if data[0][i] > data[0][j]:
            ls.append(data[0][i])
        else:
            ls.append(data[0][j])
        i=i+2
    a = np.array(ls).reshape(1,len(ls))
    return a

In [80]:
def feature_extraction(files):
    '''使用imagenet训练的resnet50抽取图片特征将本文件夹中所有的图片特征堆叠
    输入：图片的文件名列表
    输出：np.array,shape(m,1024),m为图片数量。
    '''
    model = ResNet50(weights='imagenet', include_top=False,pooling='max')
    for i in range(len(files)):
        img_path = files[i]
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        pre_features = model.predict(x)
        features = max_pool(pre_features)
        if data_num == 0:
            pre_data = features
        else:
            #垂直将数组堆叠起来，顺序与files里的图片路径对应
            pre_data = np.vstack((pre_data,features))
        print ("%3d done  total : %d" %(i+1,len(files)))
    return pre_data

In [81]:
pre_data = feature_extraction(files)

checked



In [82]:
def save_data(content,filename):
    '''保存数据为json文件
       输入：content：np.array
            filename:存储的文件名
    '''
    content = content.tolist()
    with open(filename,'w') as file_object:
        json.dump(content,file_object)

In [83]:
#保存抽取的特征
save_data(pre_data,os.path.join(class_path,'features.json'))

In [84]:
def load_data(filename):
    '''加载保存的json文件
       输入：文件名
       输出：np.array
    '''
    with open(filename,'r') as file_object:
        content = json.load(file_object)
    content = np.array(content)
    return content

In [85]:
#计算样本间距离，采用‘euclidean’
Y = sch.distance.pdist(pre_data,'euclidean')

In [86]:
#保存距离文件
save_data(Y,os.path.join(class_path,'dist.json'))

In [87]:
#计算层次聚类树，保存linkage文件和树的图片
Z=sch.linkage(Y,method='average')
save_data(Z,os.path.join(class_path,'linkage.json'))
P=sch.dendrogram(Z)
plt.savefig(os.path.join(class_path,'tree.png'))

In [90]:
#根据‘inconsistent’对树进行剪裁获得聚类标签，注意调整t值获得合适的聚类结果
labels= sch.fcluster(Z, criterion='inconsistent',t=1.1)
print (label)

In [92]:
#保存聚类结果标签，
save_data(pre_data,os.path.join(class_path,'cluster-label.json'))

In [93]:
#将聚类标签添加至文件名
for i in range(len(files)):
    os.rename(files[i],os.path.join(class_path,str(labels[i])+'-'+files[i].split('/')[-1]))   